In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn import metrics
from sklearn.metrics import accuracy_score



# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

### In this notebook, we try to implement the model LightGBM using Dask.

##### Installation of Dask-ML

In [ ]:
pip install dask-ml[complete]

In [ ]:
from dask.distributed import Client

client = Client(n_workers=4, threads_per_worker=1) # setting number of workers for parallel processing
client

In [ ]:
import dask
import xgboost
import dask_xgboost


In [ ]:
import dask_lightgbm.core as dlgbm

In [ ]:
import dask.dataframe as dd
import dask.array as da


# Load Data

In [ ]:
df_subset = dd.read_csv('../input/train-data-modified/train_df.csv')

In [ ]:
df_subset.head()

In [ ]:
test_df = dd.read_csv('../input/test-data-modified/test_df.csv')

In [ ]:
test_df.head()

##### Converting the dataframes to a Dask array.

In [ ]:
main_X = df_subset.drop(['HasDetections', 'MachineIdentifier'], axis=1).to_dask_array(lengths=True)
main_y = df_subset['HasDetections'].to_dask_array(lengths=True)

In [ ]:
test_X = test_df.drop(['MachineIdentifier'], axis=1).to_dask_array(lengths=True)

# Scaling

In [ ]:
#SCALING NOT WORKING
# from dask_ml.preprocessing import StandardScaler
# scalar = StandardScaler()
# scalar.fit_transform(main_X)

In [ ]:
import dask_ml.model_selection as dcv

In [ ]:
from dask_ml.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(main_X, main_y, test_size = 0.3, random_state = 42)

In [ ]:
# X_train

In [ ]:
# X_test

# SGD classifier

In [ ]:
from dask_ml.wrappers import Incremental
from sklearn.linear_model import SGDClassifier

In [ ]:
from dask_ml.xgboost import XGBClassifier

In [ ]:
# using the best fine-tuned hyperparameters

params = {'objective': 'binary:logistic',
          'max_depth': 4, 'eta': 0.01, 'subsample': 0.5,
          'min_child_weight': 0.5}

clf = dlgbm.LGBMClassifier()
clf.fit(main_X, main_y)

In [ ]:
y_pred = clf.predict(X_test)
print('Accuracy Score: ', (accuracy_score(y_test, y_pred)))
print('ROC-AUC score: ', metrics.roc_auc_score(y_test, y_pred))

In [ ]:
#kaggle_test_X = test_df.drop(columns="MachineIdentifier")
kaggle_test_yhat = clf.predict_proba(test_X)
#kaggle_test_yhat = clf.predict_proba(kaggle_test_X)
#kaggle_test_yhat = clf.predict_proba(kaggle_test_X)[:, -1] # predicting the probabilities
test_df["HasDetections"] = kaggle_test_yhat
to_submit = test_df[["MachineIdentifier", "HasDetections"]]
to_submit.to_csv("class-sgd.csv", single_file=True, index=False)

In [ ]:
sorted(clf.cv_results_.keys())